In [ ]:
import requests

client_id = "client_id"
client_secret = 'client_secret'

auth_url = 'https://accounts.spotify.com/api/token'
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

auth_response = requests.post(auth_url, data=data)
access_token = auth_response.json().get('access_token')
access_token

In [ ]:
import csv
import requests

# Set up your access token
access_token = access_token

# Set up the API endpoint
endpoint = "https://api.spotify.com/v1/search"

# Set up the query parameters
query_params = {
    "q": "genre:'Romantic'",
    "type": "track",
    "limit": 50  # Adjust the limit as per your needs
}

# Set up the request headers with the access token
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Send the initial GET request to the Spotify API
response = requests.get(endpoint, params=query_params, headers=headers)

# Parse the response JSON
response_json = response.json()
print(response_json)
# Extract the relevant data from the response
tracks = response_json["tracks"]["items"]

# Open the CSV file in write mode
with open("songs.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(["Song Name", "Artist(s)", "Album"])

    # Write the song details to the CSV file
    for track in tracks:
        song_name = track["name"]
        artists = ", ".join([artist["name"] for artist in track["artists"]])
        album_name = track["album"]["name"]
        writer.writerow([song_name, artists, album_name])

# Check if there are more pages of results
while response_json["tracks"]["next"]:
    next_url = response_json["tracks"]["next"]

    # Send the next GET request to the Spotify API for the next page
    response = requests.get(next_url, headers=headers)

    # Parse the response JSON
    response_json = response.json()

    # Extract the relevant data from the response
    tracks = response_json["tracks"]["items"]

    # Append the song details to the CSV file
    with open("songs.csv", mode="a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        for track in tracks:
            song_name = track["name"]
            artists = ", ".join([artist["name"] for artist in track["artists"]])
            album_name = track["album"]["name"]
            writer.writerow([song_name, artists, album_name])


In [ ]:
import pandas as pd
df = pd.read_csv('songs.csv', encoding= 'unicode_escape')

df.head()

In [ ]:
df.info()

In [ ]:
df = df.drop_duplicates(subset=['Song Name', 'Artist'], keep='first')
df.info()

In [ ]:
df.to_csv('love_songs.csv', index=False)

In [ ]:
def get_song_details(access_token, song_name, artist, album):
    search_url = 'https://api.spotify.com/v1/search'
    headers = {
        'Authorization': 'Bearer ' + access_token
    }
    params = {
        'q': f'track:"{song_name}" artist:"{artist}" album:"{album}"',
        'type': 'track',
        'limit': 1
    }
    response = requests.get(search_url, headers=headers, params=params)
    response_data = response.json()
    print(response_data)
    return response_data['tracks']['items'][0] if response_data['tracks']['items'] else None


In [ ]:
input_file = 'love_songs.csv'
output_file = 'love_songs_with_details.csv'

# Open the input and output files
with open(input_file, 'r') as csv_file, open(output_file, 'w', newline='') as output_csv:
    reader = csv.DictReader(csv_file)
    fieldnames = reader.fieldnames + ['spotify_id', 'duration_ms', 'popularity']
    writer = csv.DictWriter(output_csv, fieldnames=fieldnames)
    writer.writeheader()

    # Get the access token

    # Iterate over each song in the input file
    for row in reader:
        song_name = row['Song Name']
        artist = row['Artist']
        album = row['Album']

        # Get the song details from Spotify API
        song_details = get_song_details(access_token, song_name, artist, album)

        if song_details:
            # Extract the relevant details
            spotify_id = song_details['id']
            duration_ms = song_details['duration_ms']
            popularity = song_details['popularity']

            # Write the updated row to the output file
            row['spotify_id'] = spotify_id
            row['duration_ms'] = duration_ms
            row['popularity'] = popularity
            writer.writerow(row)


In [ ]:
def get_audio_features(access_token, track_id):
    audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
    headers = {
        'Authorization': 'Bearer ' + access_token
    }
    response = requests.get(audio_features_url, headers=headers)
    response_data = response.json()
    return response_data


In [ ]:
input_file = 'love_songs_with_details.csv'
output_file = 'love_songs_with_audio_features.csv'

# Open the input and output files
with open(input_file, 'r') as csv_file, open(output_file, 'w', newline='') as output_csv:
    reader = csv.DictReader(csv_file)
    fieldnames = reader.fieldnames + ['acousticness', 'instrumentalness', 'liveness', 'loudness', 'danceability', 'energy', 'tempo', 'valence']
    writer = csv.DictWriter(output_csv, fieldnames=fieldnames)
    writer.writeheader()


    # Iterate over each song in the input file
    for row in reader:
        song_id = row['spotify_id']

        # Get the audio features from Spotify API
        audio_features = get_audio_features(access_token, song_id)

        # Extract the relevant audio features
        acousticness = audio_features['acousticness']
        instrumentalness = audio_features['instrumentalness']
        liveness = audio_features['liveness']
        loudness = audio_features['loudness']
        danceability = audio_features['danceability']
        energy = audio_features['energy']
        tempo = audio_features['tempo']
        valence = audio_features['valence']

        # Update the row with the audio features
        row['acousticness'] = acousticness
        row['instrumentalness'] = instrumentalness
        row['liveness'] = liveness
        row['loudness'] = loudness
        row['danceability'] = danceability
        row['energy'] = energy
        row['tempo'] = tempo
        row['valence'] = valence

        # Write the updated row to the output file
        writer.writerow(row)


In [ ]:
import pickle
loaded_model = pickle.load(open('rf_model.sav', 'rb'))

df = pd.read_csv('love_songs_with_audio_features.csv', encoding= 'unicode_escape')
df = df.drop(['spotify_id', 'Artist', 'Album', 'Song Name'], axis=1)
result = loaded_model.predict(df)

#add the result to the dataframe
df['result'] = result

In [ ]:
df

In [ ]:
df.to_csv("big_train.csv", index=False)